In [ ]:
# Loading Documents in Loader

from langchain.document_loaders import TextLoader

def load_from_txt(filepath):
    loader=TextLoader(filepath,encoding='utf-8')
    return loader.load()
info=load_from_txt('./Dataset/Data.txt') # Change to path of Data file
info

In [ ]:
# Splitting Documents into chunks

from langchain.text_splitter import MarkdownHeaderTextSplitter

default_headers= [
    ("#", "Heading"),
    ("##", "Subheading"),
    ("###", "Question"),
]

def markdown_split(data,headers_split=default_headers):
    markdown_splitter=MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_split
    )
    md_header_splits=markdown_splitter.split_text(data)
    return md_header_splits

md_split=markdown_split(info[0].page_content)
md_split

In [ ]:
# Getting OpenAI embeddings 

import os
import openai
import sys
from langchain.embeddings.openai import OpenAIEmbeddings

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY'] # Provide openAI API key

def openai_embeddings(key=openai.api_key):
    return OpenAIEmbeddings(openai_api_key=key)
embeddings=openai_embeddings()

In [ ]:
# Creating Vectorspace to store the embedded data

from langchain.vectorstores import Chroma

def chroma_db(docs,embeddings):
    db = Chroma.from_documents(docs, embeddings)
    return db

db=chroma_db(md_split,embeddings)
db

In [ ]:
# Using gpt for send llm calls

from langchain.chat_models import ChatOpenAI

llm_name = "gpt-3.5-turbo" # Use required model

def llm(llm_name=llm_name, temp=0,key=openai.api_key):
    llm = ChatOpenAI(model_name=llm_name, temperature=temp,openai_api_key=key)
    return llm

llm=llm()

In [ ]:
# Making a retrival chain for extracting from vectorspace and sending calls to llm

from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA


memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible but be as informative as possible. Always say "Thanks for asking! Feel free to ask anymore Questions" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

def qa_retrival_chain(db, llm):
    qa= ConversationalRetrievalChain.from_llm(
        llm,
        chain_type="stuff", # Can use other methods for larger documents (More expensive due to more llm calls)
        retriever=db.as_retriever(),
        return_generated_question=True
    )
    return qa

def qa_chain_from_promt(db, llm, promt=QA_CHAIN_PROMPT):
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff",
        retriever=db.as_retriever(),
        chain_type_kwargs={"prompt": promt}
    )
    return qa_chain

qa=qa_retrival_chain(db,llm)

In [ ]:
# Chatbot creation with chat history for previous interactions

query=input("Enter Query \nUser:")
chat_history = []
while(1):
    if (query=="end" or query==""):
        print("Chatbot: Session Complete!")
        break
    else:
        result=qa({"question": query,"chat_history": chat_history})
        chat_history.extend([(query, result["answer"])])
        print("Chatbot:"+result['answer']+"\n")
        query=input("User: ")